In [2]:
import os
import sys
import pandas as pd

# Ajout du path pour src
sys.path.append('..')

from src.preprocessing import DataLoader
from src.models import SemanticHybridRecommender




# --- CONFIGURATION ---
DATA_DIR = '../data'
SUBMISSION_DIR = '../submission'
BEST_ALPHA = 0.5
# On utilise la configuration Ensemble qui a donné le meilleur score
BEST_HALFLIFE = [1, 250]

#BEST_HALFLIFE = [30, 150]

print(f"🚀 Lancement de la génération du fichier de soumission...")
print(f"Paramètres : Alpha={BEST_ALPHA}, Half-Lives={BEST_HALFLIFE}")

# 1. Chargement des données COMPLETES
print("Chargement des données...")
loader = DataLoader(f'{DATA_DIR}/interactions_train.csv', f'{DATA_DIR}/items_enriched_ai_turbo.csv')
full_data = loader.get_full_data()

# 2. Initialisation du Modèle
# Note : SemanticHybridRecommender remplace TimeDecayRecommender
model = SemanticHybridRecommender(loader.n_users, loader.n_items)

# 3. Entraînement
print("Entraînement du modèle sur l'ensemble des données...")
model.fit(
    full_data,
    loader.items_df,
    alpha=BEST_ALPHA,
    half_life_days=BEST_HALFLIFE,
    ##ensemble_weights=[0.6, 0.4] a fait overfité et réduit mon score kaggle

)

# 4. Prédiction
print("Calcul des scores pour tous les utilisateurs...")
# Le modèle prédit pour les utilisateurs 0 à N-1 (indices internes)
all_preds_indices = model.predict(k=10, batch_size=1000)

# 5. Génération du fichier CSV au format Kaggle
print("Lecture du fichier sample_submission.csv pour l'ordre des utilisateurs...")
try:
    sample_sub = pd.read_csv(f'{DATA_DIR}/sample_submission.csv')
    target_users = sample_sub['user_id'].unique()
except FileNotFoundError:
    print("⚠️ 'sample_submission.csv' introuvable. Prédiction pour tous les utilisateurs connus.")
    target_users = loader.u_unique

# Calcul des items populaires pour les Cold Start (utilisateurs inconnus dans le train)
top_popular = full_data['i'].value_counts().head(10).index.tolist()
top_popular_str = " ".join([str(x) for x in top_popular])

if not os.path.exists(SUBMISSION_DIR):
    os.makedirs(SUBMISSION_DIR)

output_file = f'{SUBMISSION_DIR}/submission_final.csv'

print(f"Génération du fichier {output_file}...")

with open(output_file, 'w') as f:
    f.write('user_id,recommendation\n')

    known_count = 0
    cold_count = 0

    for uid in target_users:
        # Vérifier si l'utilisateur existe dans nos mappings (Warm Start)
        if uid in loader.u_map:
            # Récupérer l'index interne
            u_idx = loader.u_map[uid]

            # Récupérer les indices des items recommandés par le modèle
            pred_row = all_preds_indices[u_idx]

            # CRUCIAL : Convertir indices internes -> Vrais IDs d'items
            pred_items = [str(loader.idx_to_i[idx]) for idx in pred_row]
            pred_str = " ".join(pred_items)
            known_count += 1
        else:
            # Cold Start (Utilisateur jamais vu) -> On recommande les populaires
            pred_str = top_popular_str
            cold_count += 1

        f.write(f"{uid},{pred_str}\n")

print("-" * 30)
print(f"🎉 Terminé !")
print(f"Utilisateurs connus (Warm) : {known_count}")
print(f"Utilisateurs inconnus (Cold) : {cold_count}")
print(f"Fichier sauvegardé : {output_file}")

🚀 Lancement de la génération du fichier de soumission...
Paramètres : Alpha=0.5, Half-Lives=[7, 365]
Chargement des données...
Entraînement du modèle sur l'ensemble des données...
Fitting SemanticHybrid Decoupled | Alpha=0.5, HL=[7, 365], Weights=[0.5, 0.5]...
Loading S-BERT & Encoding Metadata...


Batches: 100%|██████████| 473/473 [00:11<00:00, 41.75it/s]



--- Building Sub-Model 1 (Half-life=7d) ---

--- Building Sub-Model 2 (Half-life=365d) ---

Computing Global Popularity Scores...
Ensemble Model Fitted Successfully.
Calcul des scores pour tous les utilisateurs...
Lecture du fichier sample_submission.csv pour l'ordre des utilisateurs...
Génération du fichier ../submission/submission_final.csv...
------------------------------
🎉 Terminé !
Utilisateurs connus (Warm) : 7838
Utilisateurs inconnus (Cold) : 0
Fichier sauvegardé : ../submission/submission_final.csv
